In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib inline

In [ ]:
import numpy as np

from ICARUS.Vehicle.plane import Airplane as Plane
from ICARUS.Vehicle.wing import Wing as wg
import ICARUS.Vehicle.wing as wing

from ICARUS.Software.GenuVP3.angles import runGNVPangles, runGNVPanglesParallel
from ICARUS.Software.GenuVP3.pertrubations import runGNVPpertr,runGNVPpertrParallel
from ICARUS.Software.GenuVP3.pertrubations import runGNVPsensitivity,runGNVPsensitivityParallel
from ICARUS.Software.GenuVP3.filesInterface import makePolar, pertrResults
from ICARUS.Software.XFLR5.polars import readPolars2D
from ICARUS.Database.Database_2D import Database_2D
from ICARUS.Database import DB3D

In [ ]:
import os 
HOMEDIR = os.getcwd()

# Airfoil Data

In [ ]:
db = Database_2D(HOMEDIR)
airfoils = db.getAirfoils()
readPolars2D(db,HOMEDIR, f"{HOMEDIR}/ICARUS/XFLR5/")
polars2D = db.Data

# Get Plane

In [ ]:
Origin = np.array([0., 0., 0.])

In [ ]:
wingPos = np.array([0.0 - 0.159/4, 0.0, 0.0])
wingOrientation = np.array([2.8, 0.0, 0.0])

mainWing = wg(name="wing",
              airfoil=airfoils['NACA4415'],
              Origin=Origin + wingPos,
              Orientation=wingOrientation,
              isSymmetric=True,
              span=2 * 1.130,
              sweepOffset=0,
              dihAngle=0,
              chordFun=wing.linearChord,
              chord=[0.159, 0.072],
              spanFun=wing.linSpan,
              N=30,
              M=5,
              mass=0.670)
# mainWing.plotWing()

In [ ]:
elevatorPos = np.array([0.54 - 0.130/4, 0., 0.])
elevatorOrientantion = np.array([0., 0., 0.])

elevator = wg(name="tail",
              airfoil=airfoils['NACA0008'],
              Origin=Origin + elevatorPos,
              Orientation=elevatorOrientantion,
              isSymmetric=True,
              span=2 * 0.169,
              sweepOffset=0,
              dihAngle=0,
              chordFun=wing.linearChord,
              chord=[0.130, 0.03],
              spanFun=wing.linSpan,
              N=15,
              M=5,
              mass=0.06)
# elevator.plotWing()

In [ ]:
rudderPos = np.array([0.47 - 0.159/4, 0., 0.01])
rudderOrientantion = np.array([0.0, 0.0, 90.0])

rudder = wg(name="rudder",
            airfoil=airfoils['NACA0008'],
            Origin=Origin + rudderPos,
            Orientation=rudderOrientantion,
            isSymmetric=False,
            span=0.160,
            sweepOffset=0.1,
            dihAngle=0,
            chordFun=wing.linearChord,
            chord=[0.2, 0.1],
            spanFun=wing.linSpan,
            N=15,
            M=5,
            mass=0.04)
# rudder.plotWing()

In [ ]:
liftingSurfaces = [mainWing, elevator, rudder]

addedMasses = [
    (0.500 , np.array([-0.40, 0.0, 0.0])), # Motor
    (1.000 , np.array([0.090, 0.0, 0.0])), # Battery
    (0.900 , np.array([0.130, 0.0, 0.0])), # Payload
    ]
ap = Plane("Hermes5", liftingSurfaces)
ap.accessDB(HOMEDIR, DB3D)
# ap.visAirplane()
ap.addMasses(addedMasses)

In [ ]:
polarsGNVP3 = True
pertrAnalysisGNVP3 = True
sensAnalysisGNVP3 = True

## AoA Run

In [ ]:
AoAmin = -6
AoAmax = 7
NoAoA = (AoAmax - AoAmin) + 1
angles = np.linspace(AoAmin, AoAmax, NoAoA)

Uinf = 20
ap.defineSim(Uinf, 1.225)
maxiter = 2
timestep = 5e-3
# ap.save()

In [ ]:
if polarsGNVP3 == True:
    GNVP3BatchArgs = [ap, polars2D,"Xfoil",
                     maxiter,timestep,Uinf, angles]
    ap.runAnalysis(runGNVPanglesParallel, GNVP3BatchArgs)
GNVP3PolarArgs = [ap.CASEDIR, HOMEDIR]
ap.setPolars(makePolar,GNVP3PolarArgs)
ap.save()


# Dynamics

In [ ]:
from ICARUS.Flight_Dynamics.dyn_plane import dyn_Airplane as dp

### Define and Trim Plane

In [ ]:
dyn = dp(ap)

### Pertrubations

In [ ]:
dyn.allPerturb("Central")
# dyn.get_pertrub()

In [ ]:
dyn.accessDynamics(HOMEDIR)
if pertrAnalysisGNVP3 == True:
    maxiter = 2
    timestep = 5e-2
    GNVP3BatchArgs = [dyn, polars2D,"Xfoil",  maxiter, timestep,
                      dyn.trim['U'], dyn.trim['AoA']]
    dyn.runAnalysis(runGNVPpertrParallel,GNVP3BatchArgs)
GNVP3LogArgs = [dyn.DynDir, HOMEDIR]
dyn.setPertResults(pertrResults,GNVP3LogArgs)
dyn.save()

In [ ]:
if sensAnalysisGNVP3 == True:
    for var in ['u','w','q','theta','v','p','r','phi']:
        space = np.logspace(np.log10(0.00001), np.log10(1), 10, base=10)
        space =[*-space,*space]
        maxiter = 2
        timestep = 5e-2
        dyn.sensitivityAnalysis(var, space)
        GNVP3BatchArgs = [dyn, var, polars2D, "Xfoil",
                        maxiter, timestep,
                        dyn.trim['U'], dyn.trim['AoA']]
        dyn.runAnalysis(runGNVPsensitivityParallel, GNVP3BatchArgs)
        dyn.sensResults[var] = pertrResults(f"{dyn.CASEDIR}/Sensitivity_{var}", HOMEDIR)
    dyn.save()